In [15]:
import random
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta
import plotly.graph_objs as go
import pandas as pd

import random
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta
import numpy as np  # Import numpy for NaN

fake = Faker()

# Define product categories and representative product names
products = {
    "Electronics": ["Smartphone", "Laptop", "Headphones", "Camera", "Smartwatch"],
    "Fashion": ["T-shirt", "Jeans", "Sneakers", "Jacket", "Dress"],
    "Home & Kitchen": ["Blender", "Vacuum Cleaner", "Coffee Maker", "Microwave Oven", "Dishwasher"],
    "Books": ["Novel", "Biography", "Science Fiction", "Non-Fiction", "Fantasy"],
    "Sports & Outdoors": ["Tennis Racket", "Yoga Mat", "Dumbbells", "Bicycle", "Running Shoes"],
    "Beauty & Personal Care": ["Shampoo", "Lipstick", "Moisturizer", "Perfume", "Hair Dryer"],
    "Toys & Games": ["Action Figure", "Board Game", "Puzzle", "Lego Set", "Doll"],
    "Grocery": ["Organic Milk", "Eggs", "Bread", "Chicken Breast", "Rice"],
    "Automotive": ["Car Cover", "Engine Oil", "Car Vacuum", "Tire Inflator", "Car Battery"],
    "Health & Household": ["Vitamins", "Toothpaste", "Laundry Detergent", "Disinfectant Wipes", "Toilet Paper"]
}

# Define payment methods
payment_methods = [
    "Amazon Pay UPI", "Google Pay", "PhonePe", "Credit Card", 
    "Debit Card", "EMI", "Net Banking", "Cash on Delivery" , "Amazon Pay Later"
]

# Generate random data for 50 orders
data = []

# Generate a random date within the past two years
def random_date():
    end_date = datetime.now()
    start_date = end_date - timedelta(days=370)  # Approximately two years
    return fake.date_between(start_date=start_date, end_date=end_date)

for order_id in range(1, 51):  # Change to 51 to generate 50 orders (1 to 50)
    product_category = random.choice(list(products.keys()))
    product_name = random.choice(products[product_category])
    mrp = round(random.uniform(100, 20000), 2)  # MRP between 100 and 2000
    price_for_customer = round(mrp * random.uniform(0.5, 1.0), 2)  # Price between 50% and 100% of MRP
    savings = mrp - price_for_customer
    payment_method = random.choice(payment_methods)
    order_date = random_date()
    order_month = order_date.strftime("%B")
    
    if payment_method == "EMI":
        # For EMI, generate random monthly payment amount and duration
        monthly_payment = round(price_for_customer / random.randint(3, 12), 2)  # Divide total amount by 3 to 12 months
        duration_months = random.randint(3, 12)
    elif payment_method == "Amazon Pay Later":
        tenure = 1
        if price_for_customer >= 6000: 
            tenure = 12 
        elif price_for_customer >= 4500: 
            tenure = random.choice([9, 12])
        elif price_for_customer >= 3000: 
            tenure = random.choice([6, 9, 12])
        elif price_for_customer >= 0: 
            tenure = random.choice([1, 3])
        duration_months = tenure 
        monthly_payment = price_for_customer/tenure 
    else:
        # For other payment methods, set monthly_payment and duration to NaN
        monthly_payment = np.nan
        duration_months = np.nan
    
    data.append([
        order_id, product_name, product_category, mrp, price_for_customer, savings, 
        payment_method, monthly_payment, duration_months, order_date, order_month
    ])

# Create a DataFrame
columns = ["Order ID", "Product Name", "Product Category", "MRP", "Price for Customer", 
           "Savings", "Payment Method", "Monthly Payment", "Duration (months)", "Order Date", "Order Month"]

df = pd.DataFrame(data, columns=columns)

# Display the DataFrame
# print(df)

# Optionally, save to a CSV file
df.to_csv("customer_orders.csv", index=False)


In [16]:

def plot_spending_by_category(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]
    
    category_spending = filtered_df.groupby('Product Category')['Price for Customer'].sum().reset_index()
    spending_figure = {
        'data': [
            {'y': category_spending['Product Category'].tolist(), 'x': category_spending['Price for Customer'].tolist(), 'type': 'bar', 'orientation': 'h', 'name': 'Spending'}
        ],
        'layout': {
            'title': 'Spending by Category',
            'yaxis': {'title': 'Category', 'automargin': True},
            'xaxis': {'title': 'Total Spending'},
            'margin': {'l': 150, 'r': 20, 't': 50, 'b': 50}
        }
    }
    return spending_figure

def plot_savings_by_category(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]
    
    category_savings = filtered_df.groupby('Product Category')['Savings'].sum().reset_index()
    savings_figure = {
        'data': [
            {'y': category_savings['Product Category'].tolist(), 'x': category_savings['Savings'].tolist(), 'type': 'bar', 'orientation': 'h', 'name': 'Savings'}
        ],
        'layout': {
            'title': 'Savings by Category',
            'yaxis': {'title': 'Category', 'automargin': True},
            'xaxis': {'title': 'Total Savings'},
            'margin': {'l': 150, 'r': 20, 't': 50, 'b': 50}
        }
    }
    return savings_figure

def create_total_spending_card(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]
    
    total_spending = filtered_df['Price for Customer'].sum()
    card = {
        'title': 'Total Spending',
        'value': f'₹{total_spending:,.2f}'
    }
    return card

def create_total_savings_card(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]
    
    total_savings = filtered_df['Savings'].sum()
    card = {
        'title': 'Total Savings',
        'value': f'₹{total_savings:,.2f}'
    }
    return card

# Example usage:
filtered_df = df.copy()  # Load your filtered DataFrame here
selected_year = 'Overall'
selected_month = 'Overall'

spending_fig = plot_spending_by_category(filtered_df, selected_year, selected_month)
savings_fig = plot_savings_by_category(filtered_df, selected_year, selected_month)
total_spending_card = create_total_spending_card(filtered_df, selected_year, selected_month)
total_savings_card = create_total_savings_card(filtered_df, selected_year, selected_month)

print(spending_fig)
print(savings_fig)
print(total_spending_card)
print(total_savings_card)


{'data': [{'y': ['Automotive', 'Beauty & Personal Care', 'Electronics', 'Fashion', 'Grocery', 'Health & Household', 'Home & Kitchen', 'Sports & Outdoors', 'Toys & Games'], 'x': [55839.89, 48788.99, 38319.76, 16121.45, 48817.04, 14963.779999999999, 45188.72, 100900.28, 50991.24], 'type': 'bar', 'orientation': 'h', 'name': 'Spending'}], 'layout': {'title': 'Spending by Category', 'yaxis': {'title': 'Category', 'automargin': True}, 'xaxis': {'title': 'Total Spending'}, 'margin': {'l': 150, 'r': 20, 't': 50, 'b': 50}}}
{'data': [{'y': ['Automotive', 'Beauty & Personal Care', 'Electronics', 'Fashion', 'Grocery', 'Health & Household', 'Home & Kitchen', 'Sports & Outdoors', 'Toys & Games'], 'x': [29313.820000000003, 18895.75, 8336.369999999999, 4949.670000000001, 24426.33, 6870.800000000001, 14781.579999999998, 36163.56, 14433.110000000004], 'type': 'bar', 'orientation': 'h', 'name': 'Savings'}], 'layout': {'title': 'Savings by Category', 'yaxis': {'title': 'Category', 'automargin': True}, 'x

In [17]:


def plot_spending_by_category(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]

    category_spending = filtered_df.groupby('Product Category')['Price for Customer'].sum().reset_index()
    category_spending = category_spending.sort_values(by='Price for Customer', ascending=False)
    
    trace = go.Bar(
        x=category_spending['Price for Customer'],
        y=category_spending['Product Category'],
        orientation='h',
        marker=dict(color='rgba(50, 171, 96, 0.6)'),
        text=category_spending['Price for Customer'].apply(lambda x: f"₹{x:,.2f}"),
        textposition='auto'
    )
    
    layout = go.Layout(
        title='Spending by Category',
        xaxis=dict(title='Total Spending'),
        yaxis=dict(title='Category', automargin=True),
        margin=dict(l=150, r=20, t=50, b=50),
        showlegend=False
    )
    
    fig = go.Figure(data=[trace], layout=layout)
    return fig

def plot_savings_by_category(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]
    
    category_savings = filtered_df.groupby('Product Category')['Savings'].sum().reset_index()
    category_savings = category_savings.sort_values(by='Savings', ascending=False)
    
    trace = go.Bar(
        x=category_savings['Savings'],
        y=category_savings['Product Category'],
        orientation='h',
        marker=dict(color='rgba(219, 64, 82, 0.6)'),
        text=category_savings['Savings'].apply(lambda x: f"₹{x:,.2f}"),
        textposition='auto'
    )
    
    layout = go.Layout(
        title='Savings by Category',
        xaxis=dict(title='Total Savings'),
        yaxis=dict(title='Category', automargin=True),
        margin=dict(l=150, r=20, t=50, b=50),
        showlegend=False
    )
    
    fig = go.Figure(data=[trace], layout=layout)
    return fig

filtered_df = df.copy()  
selected_year = 'Overall'
selected_month = 'Overall'

spending_fig = plot_spending_by_category(filtered_df, selected_year=selected_year, selected_month=selected_month)
savings_fig = plot_savings_by_category(filtered_df, selected_year=selected_year, selected_month=selected_month)

spending_fig.show()
print()
savings_fig.show()


In [18]:

def plot_spending_over_months(filtered_df, selected_year='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[pd.to_datetime(filtered_df['Order Date']).dt.year == int(selected_year)]

    monthly_spending = filtered_df.groupby(pd.to_datetime(filtered_df['Order Date']).dt.to_period('M'))['Price for Customer'].sum().reset_index()
    monthly_spending['Date'] = monthly_spending['Order Date'].dt.strftime('%Y-%m')  # Convert to YYYY-MM format
    monthly_spending = monthly_spending.sort_values('Date')

    trace = go.Scatter(
        x=monthly_spending['Date'],
        y=monthly_spending['Price for Customer'],
        mode='lines+markers',
        name='Spending',
        line=dict(color='rgb(50, 171, 96)'),
        marker=dict(symbol='circle', size=8),
        text=monthly_spending['Price for Customer'].apply(lambda x: f"₹{x:,.2f}")
    )

    layout = go.Layout(
        title='Monthly Spending',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Total Spending'),
        margin=dict(l=150, r=20, t=50, b=50),
        showlegend=True
    )

    fig = go.Figure(data=[trace], layout=layout)
    return fig

def plot_savings_over_months(filtered_df, selected_year='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[pd.to_datetime(filtered_df['Order Date']).dt.year == int(selected_year)]

    monthly_savings = filtered_df.groupby(pd.to_datetime(filtered_df['Order Date']).dt.to_period('M'))['Savings'].sum().reset_index()
    monthly_savings['Date'] = monthly_savings['Order Date'].dt.strftime('%Y-%m')  # Convert to YYYY-MM format
    monthly_savings = monthly_savings.sort_values('Date')

    trace = go.Scatter(
        x=monthly_savings['Date'],
        y=monthly_savings['Savings'],
        mode='lines+markers',
        name='Savings',
        line=dict(color='rgb(219, 64, 82)'),
        marker=dict(symbol='circle', size=8),
        text=monthly_savings['Savings'].apply(lambda x: f"₹{x:,.2f}")
    )

    layout = go.Layout(
        title='Monthly Savings',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Total Savings'),
        margin=dict(l=150, r=20, t=50, b=50),
        showlegend=True
    )

    fig = go.Figure(data=[trace], layout=layout)
    return fig

selected_year = '2024'  

spending_over_months_fig = plot_spending_over_months(df, selected_year=selected_year)
savings_over_months_fig = plot_savings_over_months(df, selected_year=selected_year)

spending_over_months_fig.show()
savings_over_months_fig.show()


In [19]:
df

,Order ID,Product Name,Product Category,MRP,Price for Customer,Savings,Payment Method,Monthly Payment,Duration (months),Order Date,Order Month
0,1,Dumbbells,Sports & Outdoors,13355.43,7337.81,6017.62,Debit Card,NaN,NaN,2023-09-22,September
1,2,Car Cover,Automotive,15776.85,11118.26,4658.59,EMI,926.520000,3.0,2024-06-22,June
2,3,Blender,Home & Kitchen,13996.98,9226.18,4770.80,Amazon Pay UPI,NaN,NaN,2024-01-04,January
3,4,Puzzle,Toys & Games,4130.69,3259.66,871.03,Net Banking,NaN,NaN,2024-03-31,March
4,5,Tennis Racket,Sports & Outdoors,9596.21,4893.04,4703.17,Net Banking,NaN,NaN,2023-06-26,June
5,6,Chicken Breast,Grocery,9174.61,6389.63,2784.98,Cash on Delivery,NaN,NaN,2023-09-08,September
6,7,Dumbbells,Sports & Outdoors,5912.16,5872.95,39.21,Credit Card,NaN,NaN,2023-10-24,October
7,8,Tire Inflator,Automotive,9483.51,5680.20,3803.31,Google Pay,NaN,NaN,2023-11-17,November
8,9,Yoga Mat,Sports & Outdoors,16438.55,12353.06,4085.49,EMI,1372.560000,11.0,2023-08-22,August
9,10,Bread,Grocery,3340.18,2296.30,1043.88,Debit Card,NaN,NaN,2024-01-10,January


In [20]:
from datetime import datetime
def get_amazon_pay_later_details(df): 
    
    current_date = datetime.now()
    apl_df = df[df['Payment Method'] == 'Amazon Pay Later'].copy()

    # if apl_df.empty:
    #     st.info("No Amazon Pay Later orders found.")
    #     return

    # Calculate apl details
    current_date = datetime.now()
    apl_df['Duration (months)'] = apl_df['Duration (months)'].fillna(0).astype(int)
    apl_df['Monthly Payment'] = apl_df['Monthly Payment'].fillna(0).astype(float)
    apl_df['Order Date'] = pd.to_datetime(apl_df['Order Date'])
    apl_df['Months Passed'] = apl_df['Order Date'].apply(lambda x: (current_date.year - x.year) * 12 + current_date.month - x.month)

    apl_df['Amount Paid'] = apl_df.apply(lambda row: min(row['Months Passed'], row['Duration (months)']) * row['Monthly Payment'], axis=1)
    apl_df['Amount Left'] = apl_df['Price for Customer'] - apl_df['Amount Paid']

    return apl_df

get_amazon_pay_later_details(df)

,Order ID,Product Name,Product Category,MRP,Price for Customer,Savings,Payment Method,Monthly Payment,Duration (months),Order Date,Order Month,Months Passed,Amount Paid,Amount Left
16,17,Tennis Racket,Sports & Outdoors,18009.92,15331.88,2678.04,Amazon Pay Later,1277.656667,12,2023-11-20,November,7,8943.596667,6388.283333
20,21,Blender,Home & Kitchen,4109.63,2304.48,1805.15,Amazon Pay Later,2304.480000,1,2023-09-17,September,9,2304.480000,0.000000
21,22,Moisturizer,Beauty & Personal Care,19119.14,13987.96,5131.18,Amazon Pay Later,1165.663333,12,2023-07-14,July,11,12822.296667,1165.663333
23,24,Organic Milk,Grocery,15663.52,9612.10,6051.42,Amazon Pay Later,801.008333,12,2024-06-05,June,0,0.000000,9612.100000
25,26,Vitamins,Health & Household,9975.27,6398.23,3577.04,Amazon Pay Later,533.185833,12,2023-08-20,August,10,5331.858333,1066.371667
28,29,Running Shoes,Sports & Outdoors,15303.95,12963.82,2340.13,Amazon Pay Later,1080.318333,12,2024-05-26,May,1,1080.318333,11883.501667
30,31,Jeans,Fashion,10235.76,10195.25,40.51,Amazon Pay Later,849.604167,12,2023-07-23,July,11,9345.645833,849.604167
33,34,Car Battery,Automotive,9832.14,5678.15,4153.99,Amazon Pay Later,473.179167,12,2023-11-10,November,7,3312.254167,2365.895833
41,42,Bicycle,Sports & Outdoors,17516.32,11822.29,5694.03,Amazon Pay Later,985.190833,12,2024-02-07,February,4,3940.763333,7881.526667
